In [1]:
import torch
import polars as pl
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

In [3]:
train_dataset = torch.load('./train.pt').dataset
test_dataset = torch.load('./test.pt').dataset

train_loader = DataLoader([(x['image'].numpy().reshape(3, 500, 500), x['label'].int()) for x in iter(train_dataset)], batch_size=32, shuffle=True)
test_loader =  DataLoader([(x['image'].numpy().reshape(3, 500, 500), x['label'].int()) for x in iter(test_dataset)], batch_size=32, shuffle=False)

/var/folders/4w/fhdcccr162j9j3w94494tdnh0000gn/T/ipykernel_26127/3335847947.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load('./train.pt').data

# Define Neural Network Model

In [4]:
class BaselineNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size=500, num_classes=5):
        # define the layers of the network
        super(BaselineNeuralNetwork, self).__init__()
        # 3 input channels: RGB
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = torch.nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=13, kernel_size=5)
        self.fc1 = torch.nn.Linear(in_features=13 * 122 * 122, out_features=120)
        self.fc2 = torch.nn.Linear(in_features=120, out_features=84)
        # final output channels = # classes
        self.fc3 = torch.nn.Linear(in_features=84, out_features=5)

    def forward(self, x):
        x = x.float()
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, self.fc1.in_features)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = BaselineNeuralNetwork().to(device)

# Test Function
def test(model, loader):
    # Testing loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.long().to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = (correct / total) * 100
    return accuracy


# Train Loop Setup
loss_fn = torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
epochs = 3

# Train loop
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch + 1, epochs))
    model.train()
    for inputs, labels in tqdm(train_loader):
        # inputs = torch.from_numpy(inputs)
        inputs = inputs.to(device)
        labels = labels.long().to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.long())
        loss.backward()
        optimizer.step()
    accuracy = test(model, test_loader)
    print('Epoch {}, Test Accuracy: {:.2f}%'.format(epoch+1, accuracy))

Epoch 1/3


100%|██████████| 285/285 [00:36<00:00,  7.87it/s]


Epoch 1, Test Accuracy: 49.73%
Epoch 2/3


100%|██████████| 285/285 [00:35<00:00,  7.92it/s]


Epoch 2, Test Accuracy: 55.46%
Epoch 3/3


100%|██████████| 285/285 [00:35<00:00,  8.01it/s]


Epoch 3, Test Accuracy: 58.21%


In [5]:
torch.save(model.state_dict(), './baseline_nn.pt')